In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


## Data

In [2]:
def prepare_data(image_size, random_state=1):
    labels = pd.read_csv('./data/labels.csv')
    sample_submission = pd.read_csv('./data/sample_submission.csv')
    print('Train data size:', labels.shape[0])
    print('Test data size:', sample_submission.shape[0])
    labels_one_hot = np.array(pd.get_dummies(labels['breed'], sparse=True))
    
    x_train = []
    y_train = []
    x_test = []
    
    for i, id in enumerate(labels['id']):
        image = np.array(ndimage.imread('./data/train/' + id + '.jpg', flatten=False))
        x_train.append(scipy.misc.imresize(image, size=(image_size, image_size)))
        y_train.append(labels_one_hot[i])
        if (i % 1000 == 0) and (i != 0):
            print(i, 'images processed')

    for i, id in enumerate(sample_submission['id']):
        image = np.array(ndimage.imread('./data/test/' + id + '.jpg', flatten=False))
        x_test.append(scipy.misc.imresize(image, size=(image_size, image_size)))
        if (i % 1000 == 0) and (i != 0):
            print(i, 'images processed')
                
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    
    X_train, X_dev, Y_train, Y_dev = train_test_split(x_train, y_train, test_size=0.3, random_state=random_state)
                
    np.save('./data/raw/X_train', X_train)
    np.save('./data/raw/Y_train', Y_train)
    np.save('./data/raw/X_dev', X_dev)
    np.save('./data/raw/Y_dev', Y_dev)
    np.save('./data/raw/x_test', x_test)

    print('Train dataset:', x_train.shape)
    print('Train labels:', y_train.shape)
    print('Test dataset:', x_test.shape)
    
    return None

def load (name, normalize):
    data = np.load('./data/raw/' + name + '.npy')
    if normalize == True:
        data = data / 255
    return data

In [3]:
image_size = 90
input_size = (image_size, image_size, 3)
num_class = 120

In [6]:
#execute only once
prepare_data(image_size)

Train data size: 10222
Test data size: 10357


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
Train dataset: (10222, 90, 90, 3)
Train labels: (10222, 120)
Test dataset: (10357, 90, 90, 3)


In [4]:
X_train = load('X_train', normalize=True)
Y_train = load('Y_train', normalize=False)

## AlexNet

In [6]:
X_input = Input(input_size)

X = Conv2D(96, (11, 11), strides=(4,4), activation='relu')(X_input)
X = MaxPooling2D((3,3), strides=(2,2))(X)
X = Conv2D(256, (5,5), padding='same', activation='relu')(X)
X = MaxPooling2D((3,3), strides=(2,2))(X)
X = Conv2D(384, (3,3), padding='same', activation='relu')(X)
X = Conv2D(384, (3,3), padding='same', activation='relu')(X)
X = Conv2D(256, (3,3), padding='same', activation='relu')(X)
X = MaxPooling2D((3,3), strides=(2,2))(X)

X = Flatten()(X)
X = Dense(1024, activation='relu')(X)
X = Dense(512, activation='relu')(X)
output = Dense(120, activation='softmax')(X)

model = Model(inputs=X_input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train,
          batch_size=512,
          epochs=100,
          verbose=1)

Epoch 1/100
7155/7155 [==============================] - 97s 14ms/step - loss: 4.7891 - acc: 0.0067
Epoch 2/100
7155/7155 [==============================] - 96s 13ms/step - loss: 4.7824 - acc: 0.0109
Epoch 3/100
 512/7155 [=>............................] - ETA: 1:22 - loss: 4.7828 - acc: 0.0059

## VGG16 Transfer

In [5]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_size)

X = base_model.output
X = Flatten()(X)
output= Dense(num_class, activation='softmax')(X)

model = Model(inputs=base_model.inputs, outputs=output)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 22, 22, 128)       0         
__________

In [6]:
model.fit(X_train, Y_train,
          batch_size=32,
          epochs=20,
          verbose=1)

Epoch 1/20
7155/7155 [==============================] - 540s 76ms/step - loss: 4.5743 - acc: 0.0514
Epoch 2/20
1216/7155 [====>.........................] - ETA: 7:23 - loss: 3.6113 - acc: 0.2146

KeyboardInterrupt: 